# ETL Project 

## EXTRACT
Web Scraping in Sephora

In [57]:
# Dependencies 
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import time
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Initialize PyMongo to work with MongoDBs
connect = 'mongodb://localhost:27017'
client = pymongo.MongoClient(connect)

In [4]:
# Define database and collection
db = client.db_sephora
col_category = db.col_category
col_eyelip = db.col_eyelip
col_cleanser = db.col_cleanser
col_mask = db.col_mask
col_moisttreat = db.col_moisttreat
col_suncare = db.col_suncare

In [187]:
# Execute path with Chrome browser 
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# URL of webpage to be scraped
# eye_url = 'https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=300'
# lip_url = 'https://www.sephora.com/shop/lip-treatments?pageSize=300'
# cleanser1_url = 'https://www.sephora.com/shop/cleanser?pageSize=60&currentPage=9'
# cleanser2_url = 'https://www.sephora.com/shop/cleanser?pageSize=300&currentPage=2'
# mask1_url = 'https://www.sephora.com/shop/face-mask?pageSize=60&currentPage=6'
# mask2_url = 'https://www.sephora.com/shop/face-mask?pageSize=300&currentPage=2'
# moist1_url = 'https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=60&currentPage=14'
# moist2_url = 'https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=300&currentPage=2'
# moist3_url = 'https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=300&currentPage=3'
# treat1_url = 'https://www.sephora.com/shop/facial-treatments?pageSize=60&currentPage=10'
# treat2_url = 'https://www.sephora.com/shop/facial-treatments?pageSize=300&currentPage=2'
suncare_url = 'https://www.sephora.com/shop/sunscreen-sun-protection?pageSize=60&currentPage=4'

#browser.visit(eye_url)
#browser.visit(lip_url)
#browser.visit(cleanser1_url)
browser.visit(suncare_url)


In [188]:
# Retrieve product URLs
sephora_url = 'https://www.sephora.com'
product_urls = []

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Retrieve the parent results 
results = soup.find_all('div', class_='css-12egk0t')
    
# Loop through returned results 
for result in results:
    # Identify and return product 
    product = result.a['aria-label']

    # Include data in an empty list
    product_urls.append(sephora_url + result.a['href'])    

    print(product_urls)

['https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product']
['https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product']
['https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product']
['https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P4

In [189]:
# Retrieve product data
prd_detail_urls = []

for urls in product_urls:
    #print(urls)
    browser.visit(urls)
    time.sleep(3)
    soup = BeautifulSoup(browser.html, 'html.parser')
    
    brand_name = soup.find('span', class_='css-euydo4').text
    product_name = soup.find('span', class_='css-0').text
    product_price = soup.find('div', class_='css-14hdny6').text
    review_num = soup.find('span', class_='css-2rg6q7').text
    avg_stars = soup.find('div', class_='css-r17a09')['aria-label']
    
    # Dictionary to be inserted as a MongoDB document
    product_dict = {
        'product_url': urls,
        'brand_name': brand_name, 
        'product_name': product_name,
        'product_price': product_price,
        'review_num': review_num,
        'avg_stars': avg_stars
    }
    
    # Include data in an empty list
    prd_detail_urls.append(product_dict)
    
    print(prd_detail_urls)

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}]
[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}]

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

...

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

[{'product_url': 'https://www.sephora.com/product/dd-cream-dermatologically-defining-bb-cream-broad-spectrum-spf-30-P381802?icid2=products grid:p381802:product', 'brand_name': 'DERMAdoctor', 'product_name': 'DD Cream Dermatologically Defining BB Cream Broad Spectrum SPF 30', 'product_price': '$40.00', 'review_num': '282 reviews', 'avg_stars': '3.5 stars'}, {'product_url': 'https://www.sephora.com/product/radiant-translucent-powder-spf-20-P432061?icid2=products grid:p432061:product', 'brand_name': 'ILIA', 'product_name': 'Radiant Translucent Powder SPF 20', 'product_price': '$34.00', 'review_num': '9 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/day-cream-broad-spectrum-sunscreen-spf-30-P386498?icid2=products grid:p386498:product', 'brand_name': 'Anthony', 'product_name': 'Day Cream Broad Spectrum Sunscreen SPF 30', 'product_price': '$34.00', 'review_num': '6 reviews', 'avg_stars': '4 stars'}, {'product_url': 'https://www.sephora.com/product/advance

In [190]:
# Store data into MongoDB
col_suncare.insert_many(prd_detail_urls)

In [1]:
# Display products in MongoDB collection
# products = db.col_eyelip.find()

# for product in products:
#     print(product)

## TRANSFORMATION

In [42]:
# Download the collections in JSON format from MongoDB and import into Pandas Dataframe
json_cleanser = '../project2sunnys_SL/sephora_db_MongoDB/col_cleanser.json'
cleanser_df = pd.read_json(json_cleanser, lines=True)
# Insert category name 
cleanser_df.insert(1, 'category_name', 'cleansers', True)
cleanser_df.head()

,_id,category_name,avg_stars,brand_name,product_name,product_price,product_url,review_num
0,{'$oid': '5cba94c2e26c8d611aa0f934'},cleansers,4.5 stars,Drunk Elephant,T.L.C. Framboos™ Glycolic Night Serum,$90.00,https://www.sephora.com/product/t-l-c-framboos...,1K reviews
1,{'$oid': '5cba94c2e26c8d611aa0f935'},cleansers,4.5 stars,Drunk Elephant,T.L.C Sukari Babyfacial Mask,$80.00,https://www.sephora.com/product/t-l-c-sukari-b...,1K reviews
2,{'$oid': '5cba94c2e26c8d611aa0f936'},cleansers,4 stars,SK-II,Facial Treatment Essence,$179.00($214.00 value),https://www.sephora.com/product/facial-treatme...,790 reviews
3,{'$oid': '5cba94c2e26c8d611aa0f937'},cleansers,4.5 stars,Fresh,Soy Face Cleanser,$38.00,https://www.sephora.com/product/soy-face-clean...,6K reviews
4,{'$oid': '5cba94c2e26c8d611aa0f938'},cleansers,4.5 stars,The Ordinary,Glycolic Acid 7% Toning Solution,$8.70,https://www.sephora.com/product/the-ordinary-d...,433 reviews


In [25]:
len(cleanser_df)

517

In [43]:
# Download the collections in JSON format from MongoDB and import into Pandas Dataframe
json_eyelip = '../project2sunnys_SL/sephora_db_MongoDB/col_eyelip.json'
eyelip_df = pd.read_json(json_eyelip, lines=True)
# Insert category name 
eyelip_df.insert(1, 'category_name', 'eyes_lips', True)
eyelip_df.head()

,_id,category_name,avg_stars,brand_name,product_name,product_price,product_url,review_num
0,{'$oid': '5cb96cc4e26c8d50530c055d'},eyes_lips,4 stars,OLEHENRIKSEN,Banana Bright Eye Crème,$38.00,https://www.sephora.com/product/banana-bright-...,2K reviews
1,{'$oid': '5cb96cc4e26c8d50530c055e'},eyes_lips,4 stars,Drunk Elephant,C-Tango Vitamin C Eye Cream,$64.00,https://www.sephora.com/product/c-tango-multiv...,469 reviews
2,{'$oid': '5cb96cc4e26c8d50530c055f'},eyes_lips,4.5 stars,Shiseido,Benefiance WrinkleResist24 Intensive Eye Conto...,$60.00,https://www.sephora.com/product/benefiance-wri...,1K reviews
3,{'$oid': '5cb96cc4e26c8d50530c0560'},eyes_lips,3.5 stars,La Mer,The Eye Concentrate,$215.00,https://www.sephora.com/product/the-eye-concen...,109 reviews
4,{'$oid': '5cb96cc4e26c8d50530c0561'},eyes_lips,4 stars,Estée Lauder,Advanced Night Repair Eye Supercharged Complex,$62.00,https://www.sephora.com/product/advanced-night...,210 reviews


In [27]:
len(eyelip_df)

381

In [44]:
# Download the collections in JSON format from MongoDB and import into Pandas Dataframe
json_mask = '../project2sunnys_SL/sephora_db_MongoDB/col_mask.json'
mask_df = pd.read_json(json_mask, lines=True)
# Insert category name 
mask_df.insert(1, 'category_name', 'masks', True)
mask_df.head()

,_id,category_name,avg_stars,brand_name,product_name,product_price,product_url,review_num
0,{'$oid': '5cbaa909e26c8d611aa0fb39'},masks,4.5 stars,Drunk Elephant,T.L.C Sukari Babyfacial Mask,$80.00,https://www.sephora.com/product/t-l-c-sukari-b...,1K reviews
1,{'$oid': '5cbaa909e26c8d611aa0fb3a'},masks,4 stars,Summer Fridays,Jet Lag Mask,$48.00,https://www.sephora.com/product/jet-lag-mask-P...,1K reviews
2,{'$oid': '5cbaa909e26c8d611aa0fb3b'},masks,4 stars,SEPHORA COLLECTION,Sleeping Mask,$4.00 $2.00,https://www.sephora.com/product/sleeping-mask-...,1K reviews
3,{'$oid': '5cbaa909e26c8d611aa0fb3c'},masks,4 stars,GLAMGLOW,SUPERMUD® Activated Charcoal Treatment Mask,$59.00,https://www.sephora.com/product/super-mud-clea...,5K reviews
4,{'$oid': '5cbaa909e26c8d611aa0fb3d'},masks,4.5 stars,SEPHORA COLLECTION,Face Mask,$6.00,https://www.sephora.com/product/face-mask-P429...,1K reviews


In [31]:
len(mask_df)

338

In [45]:
# Export the collections in JSON format from MongoDB and import into Pandas Dataframe
json_moisttreat = '../project2sunnys_SL/sephora_db_MongoDB/col_moisttreat.json'
moisttreat_df = pd.read_json(json_moisttreat, lines=True)
# Insert category name 
moisttreat_df.insert(1, 'category_name', 'moisturisers', True)
moisttreat_df.head()

,_id,category_name,avg_stars,brand_name,product_name,product_price,product_url,review_num
0,{'$oid': '5cbab5bfe26c8d611aa0fc8b'},moisturisers,4 stars,Drunk Elephant,Protini Polypeptide Moisturizer,$68.00,https://www.sephora.com/product/protini-tm-pol...,1K reviews
1,{'$oid': '5cbab5bfe26c8d611aa0fc8c'},moisturisers,4 stars,La Mer,Crème de la Mer,$175.00,https://www.sephora.com/product/creme-de-la-me...,495 reviews
2,{'$oid': '5cbab5bfe26c8d611aa0fc8d'},moisturisers,4 stars,IT Cosmetics,CC+ Cream with SPF 50+,$39.00,https://www.sephora.com/product/your-skin-but-...,2K reviews
3,{'$oid': '5cbab5bfe26c8d611aa0fc8e'},moisturisers,4 stars,Tatcha,The Water Cream,$68.00,https://www.sephora.com/product/the-water-crea...,1K reviews
4,{'$oid': '5cbab5bfe26c8d611aa0fc8f'},moisturisers,4 stars,SK-II,Facial Treatment Essence,$179.00($214.00 value),https://www.sephora.com/product/facial-treatme...,790 reviews


In [33]:
len(moisttreat_df)

1398

In [46]:
# Download the collections in JSON format from MongoDB and import into Pandas Dataframe
json_suncare = '../project2sunnys_SL/sephora_db_MongoDB/col_suncare.json'
suncare_df = pd.read_json(json_suncare, lines=True)
# Insert category name 
suncare_df.insert(1, 'category_name', 'sun_care', True)
suncare_df.head()

,_id,category_name,avg_stars,brand_name,product_name,product_price,product_url,review_num
0,{'$oid': '5cbb41c8e26c8d611aa10201'},sun_care,4 stars,IT Cosmetics,CC+ Cream with SPF 50+,$39.00,https://www.sephora.com/product/your-skin-but-...,2K reviews
1,{'$oid': '5cbb41c8e26c8d611aa10202'},sun_care,4.5 stars,Supergoop!,Unseen Sunscreen Broad Spectrum SPF 40,$32.00,https://www.sephora.com/product/unseen-sunscre...,548 reviews
2,{'$oid': '5cbb41c8e26c8d611aa10203'},sun_care,4.5 stars,Laura Mercier,Tinted Moisturizer Broad Spectrum SPF 20,$46.00,https://www.sephora.com/product/tinted-moistur...,4K reviews
3,{'$oid': '5cbb41c8e26c8d611aa10204'},sun_care,4.5 stars,NARS,Pure Radiant Tinted Moisturizer Broad Spectrum...,$45.00,https://www.sephora.com/product/pure-radiant-t...,2K reviews
4,{'$oid': '5cbb41c8e26c8d611aa10205'},sun_care,4.5 stars,Shiseido,Urban Environment Oil-Free UV Protector Broad ...,$35.00,https://www.sephora.com/product/urban-environm...,999 reviews


In [35]:
len(suncare_df)

218

In [47]:
# Combine all product details into one dataframe
prd_detail_df = cleanser_df.append(eyelip_df)
prd_detail_df1 = prd_detail_df.append(mask_df)
prd_detail_df2 = prd_detail_df1.append(moisttreat_df)
prd_detail_df3 = prd_detail_df2.append(suncare_df)
len(prd_detail_df3)

2852

In [49]:
# Show all product details in one dataframe
prd_detail_df3.head()

,_id,category_name,avg_stars,brand_name,product_name,product_price,product_url,review_num
0,{'$oid': '5cba94c2e26c8d611aa0f934'},cleansers,4.5 stars,Drunk Elephant,T.L.C. Framboos™ Glycolic Night Serum,$90.00,https://www.sephora.com/product/t-l-c-framboos...,1K reviews
1,{'$oid': '5cba94c2e26c8d611aa0f935'},cleansers,4.5 stars,Drunk Elephant,T.L.C Sukari Babyfacial Mask,$80.00,https://www.sephora.com/product/t-l-c-sukari-b...,1K reviews
2,{'$oid': '5cba94c2e26c8d611aa0f936'},cleansers,4 stars,SK-II,Facial Treatment Essence,$179.00($214.00 value),https://www.sephora.com/product/facial-treatme...,790 reviews
3,{'$oid': '5cba94c2e26c8d611aa0f937'},cleansers,4.5 stars,Fresh,Soy Face Cleanser,$38.00,https://www.sephora.com/product/soy-face-clean...,6K reviews
4,{'$oid': '5cba94c2e26c8d611aa0f938'},cleansers,4.5 stars,The Ordinary,Glycolic Acid 7% Toning Solution,$8.70,https://www.sephora.com/product/the-ordinary-d...,433 reviews


In [50]:
# Create a filtered dataframe with specific columns 
prd_detail_df4 = prd_detail_df3[['category_name', 'avg_stars', 'brand_name', 'product_name', 'product_price', 'product_url', 'review_num']].copy()
prd_detail_df4.head()

,category_name,avg_stars,brand_name,product_name,product_price,product_url,review_num
0,cleansers,4.5 stars,Drunk Elephant,T.L.C. Framboos™ Glycolic Night Serum,$90.00,https://www.sephora.com/product/t-l-c-framboos...,1K reviews
1,cleansers,4.5 stars,Drunk Elephant,T.L.C Sukari Babyfacial Mask,$80.00,https://www.sephora.com/product/t-l-c-sukari-b...,1K reviews
2,cleansers,4 stars,SK-II,Facial Treatment Essence,$179.00($214.00 value),https://www.sephora.com/product/facial-treatme...,790 reviews
3,cleansers,4.5 stars,Fresh,Soy Face Cleanser,$38.00,https://www.sephora.com/product/soy-face-clean...,6K reviews
4,cleansers,4.5 stars,The Ordinary,Glycolic Acid 7% Toning Solution,$8.70,https://www.sephora.com/product/the-ordinary-d...,433 reviews


In [51]:
# Clean the data by dropping duplicates
prd_detail_df4.drop_duplicates('product_url', inplace=True)
len(prd_detail_df4)

2525

In [56]:
# Clean the data by removing units 
prd_detail_df4['avg_stars'] = prd_detail_df4['avg_stars'].str.replace('stars', '')
prd_detail_df4['review_num'] = prd_detail_df4['review_num'].str.replace('reviews', '')
prd_detail_df4['review_num'] = prd_detail_df4['review_num'].str.replace('K', '000')
prd_detail_df4['product_price'] = prd_detail_df4['product_price'].str.replace('$', '')
prd_detail_df4.head()

,category_name,avg_stars,brand_name,product_name,product_price,product_url,review_num
0,cleansers,4.5,Drunk Elephant,T.L.C. Framboos™ Glycolic Night Serum,90.00,https://www.sephora.com/product/t-l-c-framboos...,1000
1,cleansers,4.5,Drunk Elephant,T.L.C Sukari Babyfacial Mask,80.00,https://www.sephora.com/product/t-l-c-sukari-b...,1000
2,cleansers,4,SK-II,Facial Treatment Essence,179.00(214.00 value),https://www.sephora.com/product/facial-treatme...,790
3,cleansers,4.5,Fresh,Soy Face Cleanser,38.00,https://www.sephora.com/product/soy-face-clean...,6000
4,cleansers,4.5,The Ordinary,Glycolic Acid 7% Toning Solution,8.70,https://www.sephora.com/product/the-ordinary-d...,433


## LOAD 
Load data into MySQL

In [115]:
# Connect to local MySQL database
connection = 'root:password@localhost/skincare_db'
#engine = create_engine(f'mysql://{connection}')
engine = create_engine(f'mysql://{connection}?charset=utf8mb4')


In [116]:
# Confirm tables
engine.table_names()

['brand', 'category', 'data_source', 'product', 'review']

In [ ]:
# Load dataframe into database
prd_detail_df4.to_sql(name='product', con=engine, if_exists='append', index=False)